### Урок3. Связь бизнес-показателей и DS-метрик

План занятия:
0. Ликбез: train/test/val и crossvalidation
1. переход от вероятностей к меткам в задаче бинарной классификации
2. какие бывают DS-метрики, их интерпретация
3. пример из практики
4. бизнес-метрики
5. взаимосвязь DS и бизнес метрик
6. case1 - фильтрация токсичных комментариев
7. case2 - определение ССЗ 

In [209]:
from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

warnings.simplefilter('ignore')

In [134]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split
from scipy.sparse import hstack
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import roc_auc_score, log_loss


### Кейс 2 - определение ССЗ

ссылка на соревнование - https://mlbootcamp.ru/ru/round/12/sandbox/

В рамках конкурса вам нужно предсказать наличие сердечно-сосудистых заболеваний по результатам классического врачебного осмотра. Датасет сформирован из 100.000 реальных клинических анализов, и в нём используются признаки, которые можно разбить на 3 группы:

 

Объективные признаки:

 - Возраст
 - Рост
 - Вес
 - Пол
 

Результаты измерения:

 - Артериальное давление верхнее и нижнее
 - Холестерин
 - Глюкоза
 

Субъективные признаки:

 - Курение
 - Употребление Алкоголя
 - Физическая активность
 

Возраст дан в днях. Значения показателей холестерина и глюкозы представлены одним из трех классов: норма, выше нормы, значительно выше нормы. Значения субъективных признаков — бинарны.

Все показатели даны на момент осмотра.

Таргет - наличие сердечно-сосудистых заболеваний (ССЗ)

In [135]:
df = pd.read_csv('train_case2.csv', ';')
df.head(3)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1


Разделим наши данные на тренировочную и тестовую выборки

In [136]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(df.drop('cardio', 1), 
                                                    df['cardio'], random_state=0)

К полям:
- gender, cholesterol применим OHE-кодирование
- age, height, weight, ap_hi, ap_lo - standardScaler
- gluc, smoke, alco, active - оставим пока как есть

In [137]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]


from sklearn.preprocessing import StandardScaler


continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']

continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

Теперь объединим все наши трансформеры с помощью FeatureUnion

In [138]:
from sklearn.pipeline import FeatureUnion

feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)
feature_processing = Pipeline([('feats', feats)])

feature_processing.fit_transform(X_train)

array([[-1.73391771,  0.6873301 ,  0.74843904, ...,  1.        ,
         0.        ,  1.        ],
       [-1.67343538,  0.07758923, -0.29640123, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.13738132,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.17775864,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  1.        ],
       [-0.47190715, -1.38578883,  0.74843904, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.38174619,  0.56538192, -0.08743318, ...,  0.        ,
         0.        ,  1.        ]])

Добавим классификатор и запустим кросс-валидацию

In [139]:
classifiers = [GaussianNB(),\
               LogisticRegression(random_state = 123),\
               CatBoostClassifier(random_state=123),\
               RandomForestClassifier(max_depth=10,random_state=123),\
               GradientBoostingClassifier (random_state=123)]
results = []

In [140]:
for classifier in classifiers:
    model = Pipeline([
        ('features', feats),
        ('classifier', classifier),
    ])


    #запустим кросс-валидацию
    cv_scores = cross_val_score(model, X_train, y_train, cv=10, scoring='roc_auc')
    cv_score = np.mean(cv_scores)
    cv_score_std = np.std(cv_scores)
    print(f'{classifier}, CV score = {cv_score}+-{cv_score_std}')

    #обучим пайплайн на всем тренировочном датасете
    model.fit(X_train, y_train)
    y_score = model.predict_proba(X_test)[:, 1]
    
    precision, recall, thresholds = precision_recall_curve(y_test.values, y_score)
    b=1
    f_score = (1+b**2)*(precision * recall) / (b**2*precision + recall)
    
    ix = np.argmax(f_score)
    
    f_score = f_score[ix]
    precision = precision[ix]
    recall = recall[ix]
    
    roc_auc = roc_auc_score(y_true=y_test, y_score=y_score)
    
    result = [classifier, precision, recall, roc_auc, f_score]
    results.append(result)    

GaussianNB(), CV score = 0.6910299375853622+-0.01039647239022525
LogisticRegression(random_state=123), CV score = 0.7866749255103078+-0.00652621980218306
Learning rate set to 0.053445
0:	learn: 0.6773446	total: 13.4ms	remaining: 13.4s
1:	learn: 0.6634274	total: 23.1ms	remaining: 11.6s
2:	learn: 0.6512281	total: 34.1ms	remaining: 11.3s
3:	learn: 0.6403442	total: 45.9ms	remaining: 11.4s
4:	learn: 0.6304389	total: 56.7ms	remaining: 11.3s
5:	learn: 0.6217835	total: 68.6ms	remaining: 11.4s
6:	learn: 0.6138216	total: 78.3ms	remaining: 11.1s
7:	learn: 0.6068893	total: 88.7ms	remaining: 11s
8:	learn: 0.6006509	total: 98.6ms	remaining: 10.9s
9:	learn: 0.5952495	total: 110ms	remaining: 10.9s
10:	learn: 0.5903319	total: 121ms	remaining: 10.9s
11:	learn: 0.5857301	total: 132ms	remaining: 10.9s
12:	learn: 0.5820402	total: 144ms	remaining: 11s
13:	learn: 0.5786477	total: 160ms	remaining: 11.2s
14:	learn: 0.5752225	total: 173ms	remaining: 11.3s
15:	learn: 0.5723712	total: 190ms	remaining: 11.7s
16:	l

159:	learn: 0.5317962	total: 2.1s	remaining: 11s
160:	learn: 0.5317550	total: 2.12s	remaining: 11s
161:	learn: 0.5317082	total: 2.13s	remaining: 11s
162:	learn: 0.5316576	total: 2.14s	remaining: 11s
163:	learn: 0.5316145	total: 2.15s	remaining: 11s
164:	learn: 0.5315821	total: 2.17s	remaining: 11s
165:	learn: 0.5315787	total: 2.17s	remaining: 10.9s
166:	learn: 0.5315371	total: 2.19s	remaining: 10.9s
167:	learn: 0.5314713	total: 2.2s	remaining: 10.9s
168:	learn: 0.5313984	total: 2.21s	remaining: 10.9s
169:	learn: 0.5313856	total: 2.22s	remaining: 10.9s
170:	learn: 0.5313280	total: 2.24s	remaining: 10.8s
171:	learn: 0.5312919	total: 2.25s	remaining: 10.8s
172:	learn: 0.5312564	total: 2.26s	remaining: 10.8s
173:	learn: 0.5312200	total: 2.27s	remaining: 10.8s
174:	learn: 0.5311437	total: 2.28s	remaining: 10.8s
175:	learn: 0.5310871	total: 2.3s	remaining: 10.8s
176:	learn: 0.5310323	total: 2.32s	remaining: 10.8s
177:	learn: 0.5309871	total: 2.33s	remaining: 10.8s
178:	learn: 0.5309272	total

321:	learn: 0.5245431	total: 4.22s	remaining: 8.89s
322:	learn: 0.5245088	total: 4.24s	remaining: 8.89s
323:	learn: 0.5244603	total: 4.26s	remaining: 8.88s
324:	learn: 0.5244148	total: 4.27s	remaining: 8.87s
325:	learn: 0.5243448	total: 4.29s	remaining: 8.86s
326:	learn: 0.5243047	total: 4.3s	remaining: 8.85s
327:	learn: 0.5242720	total: 4.31s	remaining: 8.83s
328:	learn: 0.5242454	total: 4.32s	remaining: 8.81s
329:	learn: 0.5241972	total: 4.33s	remaining: 8.8s
330:	learn: 0.5241897	total: 4.34s	remaining: 8.78s
331:	learn: 0.5241559	total: 4.36s	remaining: 8.77s
332:	learn: 0.5241126	total: 4.37s	remaining: 8.75s
333:	learn: 0.5240392	total: 4.38s	remaining: 8.73s
334:	learn: 0.5240094	total: 4.39s	remaining: 8.72s
335:	learn: 0.5239777	total: 4.4s	remaining: 8.7s
336:	learn: 0.5239093	total: 4.42s	remaining: 8.69s
337:	learn: 0.5238662	total: 4.43s	remaining: 8.67s
338:	learn: 0.5237993	total: 4.45s	remaining: 8.67s
339:	learn: 0.5237448	total: 4.46s	remaining: 8.66s
340:	learn: 0.52

484:	learn: 0.5179917	total: 6.31s	remaining: 6.7s
485:	learn: 0.5179713	total: 6.32s	remaining: 6.69s
486:	learn: 0.5179390	total: 6.34s	remaining: 6.68s
487:	learn: 0.5178995	total: 6.35s	remaining: 6.67s
488:	learn: 0.5178551	total: 6.37s	remaining: 6.65s
489:	learn: 0.5178180	total: 6.38s	remaining: 6.64s
490:	learn: 0.5177696	total: 6.39s	remaining: 6.62s
491:	learn: 0.5177013	total: 6.4s	remaining: 6.61s
492:	learn: 0.5176571	total: 6.41s	remaining: 6.59s
493:	learn: 0.5176093	total: 6.42s	remaining: 6.58s
494:	learn: 0.5175624	total: 6.44s	remaining: 6.57s
495:	learn: 0.5175202	total: 6.45s	remaining: 6.55s
496:	learn: 0.5174545	total: 6.46s	remaining: 6.54s
497:	learn: 0.5174195	total: 6.47s	remaining: 6.52s
498:	learn: 0.5173704	total: 6.48s	remaining: 6.51s
499:	learn: 0.5173581	total: 6.5s	remaining: 6.5s
500:	learn: 0.5173162	total: 6.51s	remaining: 6.48s
501:	learn: 0.5172756	total: 6.53s	remaining: 6.47s
502:	learn: 0.5172430	total: 6.54s	remaining: 6.46s
503:	learn: 0.51

651:	learn: 0.5120929	total: 8.4s	remaining: 4.49s
652:	learn: 0.5120457	total: 8.42s	remaining: 4.47s
653:	learn: 0.5119997	total: 8.43s	remaining: 4.46s
654:	learn: 0.5119605	total: 8.45s	remaining: 4.45s
655:	learn: 0.5119319	total: 8.46s	remaining: 4.43s
656:	learn: 0.5118845	total: 8.47s	remaining: 4.42s
657:	learn: 0.5118571	total: 8.48s	remaining: 4.41s
658:	learn: 0.5118063	total: 8.49s	remaining: 4.39s
659:	learn: 0.5117599	total: 8.51s	remaining: 4.38s
660:	learn: 0.5116978	total: 8.52s	remaining: 4.37s
661:	learn: 0.5116930	total: 8.53s	remaining: 4.36s
662:	learn: 0.5116536	total: 8.54s	remaining: 4.34s
663:	learn: 0.5116059	total: 8.55s	remaining: 4.33s
664:	learn: 0.5115723	total: 8.56s	remaining: 4.32s
665:	learn: 0.5115384	total: 8.58s	remaining: 4.3s
666:	learn: 0.5114928	total: 8.59s	remaining: 4.29s
667:	learn: 0.5114434	total: 8.6s	remaining: 4.27s
668:	learn: 0.5114345	total: 8.62s	remaining: 4.26s
669:	learn: 0.5113720	total: 8.63s	remaining: 4.25s
670:	learn: 0.5

...

257:	learn: 0.5273807	total: 3.51s	remaining: 10.1s
258:	learn: 0.5273374	total: 3.53s	remaining: 10.1s
259:	learn: 0.5272910	total: 3.55s	remaining: 10.1s
260:	learn: 0.5272524	total: 3.56s	remaining: 10.1s
261:	learn: 0.5272136	total: 3.58s	remaining: 10.1s
262:	learn: 0.5271801	total: 3.59s	remaining: 10.1s
263:	learn: 0.5271082	total: 3.6s	remaining: 10s
264:	learn: 0.5270652	total: 3.62s	remaining: 10s
265:	learn: 0.5270120	total: 3.63s	remaining: 10s
266:	learn: 0.5269778	total: 3.64s	remaining: 10s
267:	learn: 0.5269494	total: 3.65s	remaining: 9.98s
268:	learn: 0.5268989	total: 3.67s	remaining: 9.96s
269:	learn: 0.5268604	total: 3.68s	remaining: 9.95s
270:	learn: 0.5268079	total: 3.69s	remaining: 9.93s
271:	learn: 0.5267724	total: 3.7s	remaining: 9.91s
272:	learn: 0.5267595	total: 3.72s	remaining: 9.9s
273:	learn: 0.5266992	total: 3.74s	remaining: 9.9s
274:	learn: 0.5266773	total: 3.75s	remaining: 9.88s
275:	learn: 0.5266387	total: 3.77s	remaining: 9.88s
276:	learn: 0.5266000	to

424:	learn: 0.5209657	total: 5.83s	remaining: 7.89s
425:	learn: 0.5209362	total: 5.85s	remaining: 7.88s
426:	learn: 0.5209011	total: 5.87s	remaining: 7.87s
427:	learn: 0.5208504	total: 5.88s	remaining: 7.86s
428:	learn: 0.5208020	total: 5.89s	remaining: 7.84s
429:	learn: 0.5207623	total: 5.9s	remaining: 7.83s
430:	learn: 0.5207291	total: 5.92s	remaining: 7.81s
431:	learn: 0.5206786	total: 5.93s	remaining: 7.8s
432:	learn: 0.5206618	total: 5.94s	remaining: 7.78s
433:	learn: 0.5206404	total: 5.95s	remaining: 7.76s
434:	learn: 0.5205697	total: 5.97s	remaining: 7.75s
435:	learn: 0.5205407	total: 5.98s	remaining: 7.74s
436:	learn: 0.5204956	total: 5.99s	remaining: 7.72s
437:	learn: 0.5204492	total: 6s	remaining: 7.71s
438:	learn: 0.5204032	total: 6.02s	remaining: 7.69s
439:	learn: 0.5203665	total: 6.05s	remaining: 7.7s
440:	learn: 0.5203026	total: 6.07s	remaining: 7.69s
441:	learn: 0.5202621	total: 6.08s	remaining: 7.68s
442:	learn: 0.5202137	total: 6.09s	remaining: 7.66s
443:	learn: 0.5201

589:	learn: 0.5150209	total: 8.1s	remaining: 5.63s
590:	learn: 0.5149974	total: 8.12s	remaining: 5.62s
591:	learn: 0.5149676	total: 8.14s	remaining: 5.61s
592:	learn: 0.5149225	total: 8.16s	remaining: 5.6s
593:	learn: 0.5148737	total: 8.17s	remaining: 5.58s
594:	learn: 0.5148361	total: 8.18s	remaining: 5.57s
595:	learn: 0.5148058	total: 8.2s	remaining: 5.56s
596:	learn: 0.5147887	total: 8.21s	remaining: 5.54s
597:	learn: 0.5147694	total: 8.22s	remaining: 5.53s
598:	learn: 0.5147492	total: 8.24s	remaining: 5.51s
599:	learn: 0.5147275	total: 8.25s	remaining: 5.5s
600:	learn: 0.5146776	total: 8.26s	remaining: 5.48s
601:	learn: 0.5146361	total: 8.27s	remaining: 5.47s
602:	learn: 0.5146114	total: 8.29s	remaining: 5.46s
603:	learn: 0.5145743	total: 8.3s	remaining: 5.44s
604:	learn: 0.5145383	total: 8.32s	remaining: 5.43s
605:	learn: 0.5145031	total: 8.34s	remaining: 5.42s
606:	learn: 0.5144719	total: 8.35s	remaining: 5.41s
607:	learn: 0.5144155	total: 8.37s	remaining: 5.39s
608:	learn: 0.514

759:	learn: 0.5098127	total: 10.4s	remaining: 3.29s
760:	learn: 0.5097612	total: 10.4s	remaining: 3.27s
761:	learn: 0.5097269	total: 10.4s	remaining: 3.26s
762:	learn: 0.5096969	total: 10.5s	remaining: 3.25s
763:	learn: 0.5096694	total: 10.5s	remaining: 3.23s
764:	learn: 0.5096463	total: 10.5s	remaining: 3.22s
765:	learn: 0.5096048	total: 10.5s	remaining: 3.21s
766:	learn: 0.5095657	total: 10.5s	remaining: 3.19s
767:	learn: 0.5095268	total: 10.5s	remaining: 3.18s
768:	learn: 0.5094890	total: 10.5s	remaining: 3.17s
769:	learn: 0.5094356	total: 10.6s	remaining: 3.15s
770:	learn: 0.5094112	total: 10.6s	remaining: 3.14s
771:	learn: 0.5093768	total: 10.6s	remaining: 3.12s
772:	learn: 0.5093420	total: 10.6s	remaining: 3.11s
773:	learn: 0.5093251	total: 10.6s	remaining: 3.1s
774:	learn: 0.5092992	total: 10.6s	remaining: 3.08s
775:	learn: 0.5092485	total: 10.6s	remaining: 3.07s
776:	learn: 0.5092226	total: 10.7s	remaining: 3.06s
777:	learn: 0.5091887	total: 10.7s	remaining: 3.04s
778:	learn: 0

929:	learn: 0.5048100	total: 12.7s	remaining: 958ms
930:	learn: 0.5047677	total: 12.7s	remaining: 945ms
931:	learn: 0.5047412	total: 12.8s	remaining: 931ms
932:	learn: 0.5047142	total: 12.8s	remaining: 918ms
933:	learn: 0.5046827	total: 12.8s	remaining: 904ms
934:	learn: 0.5046683	total: 12.8s	remaining: 890ms
935:	learn: 0.5046524	total: 12.8s	remaining: 876ms
936:	learn: 0.5046321	total: 12.8s	remaining: 863ms
937:	learn: 0.5046025	total: 12.8s	remaining: 849ms
938:	learn: 0.5045916	total: 12.9s	remaining: 835ms
939:	learn: 0.5045577	total: 12.9s	remaining: 821ms
940:	learn: 0.5045212	total: 12.9s	remaining: 808ms
941:	learn: 0.5044890	total: 12.9s	remaining: 794ms
942:	learn: 0.5044606	total: 12.9s	remaining: 780ms
943:	learn: 0.5044348	total: 12.9s	remaining: 766ms
944:	learn: 0.5044066	total: 12.9s	remaining: 753ms
945:	learn: 0.5043577	total: 12.9s	remaining: 739ms
946:	learn: 0.5043521	total: 13s	remaining: 726ms
947:	learn: 0.5043264	total: 13s	remaining: 712ms
948:	learn: 0.50

In [201]:
df_results = pd.DataFrame(np.array(results), columns=["model", "precision", "recall", "roc_auc", "f_score"])
df_results

,model,precision,recall,roc_auc,f_score
0,GaussianNB(),0.562726,0.877995,0.687631,0.685866
1,LogisticRegression(random_state=123),0.647431,0.837558,0.784035,0.730323
2,<catboost.core.CatBoostClassifier object at 0x...,0.69467,0.789747,0.801161,0.739163
3,"(DecisionTreeClassifier(max_depth=10, max_feat...",0.696954,0.788249,0.801538,0.739796
4,([DecisionTreeRegressor(criterion='friedman_ms...,0.697848,0.788134,0.802615,0.740248


In [208]:
print('Лучшая модель:')
df_results[df_results.roc_auc == df_results.roc_auc.max()]

Лучшая модель:


,model,precision,recall,roc_auc,f_score
4,([DecisionTreeRegressor(criterion='friedman_ms...,0.697848,0.788134,0.802615,0.740248
